In [2]:
### GAN Bueno ###

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, utils
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from pathlib import Path
from torchvision.models import resnet18
from PIL import Image
import pandas as pd
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.inception import InceptionScore

# CONFIG
csv_path = "C:/Users/34629/TFG/single_label_dataset_without_H_no_O.csv"
image_size = 128
batch_size = 16
epochs = 150
latent_dim = 256
lr = 1e-4
save_dir = Path("C:/Users/34629/TFG/wgan_gp_mio")
weights_path = "C:/Users/34629/Downloads/RETFound_MAE/RETFound_cfp_weights.pth"
target_classes = ['G', 'C', 'A', 'M']
lambda_gp = 10
n_critic = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

class CustomImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.paths = df['Image_Path'].values
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
        )
        self.skip = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        return F.relu(self.block(x) + self.skip(x))

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 512 * 4 * 4)
        self.model = nn.Sequential(
            nn.Upsample(scale_factor=2),
            ResBlock(512, 256),
            nn.Upsample(scale_factor=2),
            ResBlock(256, 128),
            nn.Upsample(scale_factor=2),
            ResBlock(128, 64),
            nn.Upsample(scale_factor=2),
            ResBlock(64, 32),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 3, 3, 1, 1),
            nn.Tanh()
        )

    def forward(self, z):
        x = self.fc(z).view(z.size(0), 512, 4, 4)
        return self.model(x)

class RETFoundDiscriminator(nn.Module):
    def __init__(self, weights_path):
        super().__init__()
        self.feature_extractor = resnet18()
        self.feature_extractor.fc = nn.Identity()
        checkpoint = torch.load(weights_path, map_location='cpu', weights_only=False)
        state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
        self.feature_extractor.load_state_dict(state_dict, strict=False)
        for name, param in self.feature_extractor.named_parameters():
            param.requires_grad = name.startswith("layer4")
        self.classifier = nn.Sequential(
            nn.Linear(512, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        feat = self.feature_extractor(x)
        return self.classifier(feat)

def compute_gradient_penalty(D, real_samples, fake_samples):
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=real_samples.device)
    interpolates = (alpha * real_samples + (1 - alpha) * fake_samples).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.ones(d_interpolates.size(), device=real_samples.device, requires_grad=False)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

for class_name in target_classes:
    print(f"\\n🚀 Clase: {class_name}")
    save_subdir = save_dir / class_name
    checkpoint_dir = save_subdir / "checkpoints_wgan_gp"
    generated_dir = save_subdir / "generated_wgan_gp"
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    generated_dir.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(csv_path)
    class_df = df[df[class_name] == 1].reset_index(drop=True)
    dataset = CustomImageDataset(class_df, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    G = Generator().to(device)
    D = RETFoundDiscriminator(weights_path).to(device)
    optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=(0.0, 0.9))
    optimizer_D = optim.Adam(filter(lambda p: p.requires_grad, D.parameters()), lr=lr, betas=(0.0, 0.9))

    ema_G = Generator().to(device)
    ema_G.load_state_dict(G.state_dict())
    for param in ema_G.parameters():
        param.requires_grad = False

    fid = FrechetInceptionDistance(normalize=True).to(device)
    is_metric = InceptionScore(normalize=True).to(device)
    best_fid = float('inf')

    def update_ema():
        with torch.no_grad():
            for p_ema, p in zip(ema_G.parameters(), G.parameters()):
                p_ema.mul_(0.999).add_(p, alpha=1 - 0.999)

    for epoch in range(epochs):
        for i, real_imgs in enumerate(tqdm(dataloader)):
            real_imgs = real_imgs.to(device)
            batch_size = real_imgs.size(0)

            for _ in range(n_critic):
                z = torch.randn(batch_size, latent_dim, device=device)
                z = z / torch.norm(z, dim=1, keepdim=True)
                fake_imgs = G(z)

                real_validity = D(real_imgs)
                fake_validity = D(fake_imgs.detach())
                gp = compute_gradient_penalty(D, real_imgs, fake_imgs.detach())
                d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gp

                optimizer_D.zero_grad()
                d_loss.backward()
                optimizer_D.step()

            z = torch.randn(batch_size, latent_dim, device=device)
            z = z / torch.norm(z, dim=1, keepdim=True)
            gen_imgs = G(z)
            g_loss = -torch.mean(D(gen_imgs))

            optimizer_G.zero_grad()
            g_loss.backward()
            optimizer_G.step()
            update_ema()

        ema_G.eval()
        with torch.no_grad():
            z = torch.randn(64, latent_dim, device=device)
            z = z / torch.norm(z, dim=1, keepdim=True)
            gen_imgs = ema_G(z)

            utils.save_image(gen_imgs, generated_dir / f"epoch_{epoch:03d}.png", nrow=8, normalize=True)
            torch.save(ema_G.state_dict(), checkpoint_dir / f"WGAN_ema_G_epoch_{epoch:03d}.pth")

            fid.update(gen_imgs, real=False)
            fid.update(real_imgs[:64], real=True)
            is_metric.update(gen_imgs)

            fid_score = fid.compute().item()
            is_score = is_metric.compute()[0].item()
            print(f"[Epoch {epoch+1}] FID: {fid_score:.2f}, IS: {is_score:.2f}")

            if fid_score < best_fid:
                best_fid = fid_score
                torch.save(ema_G.state_dict(), checkpoint_dir / f"best_WGAN_ema_G_{class_name}.pth")
                print("✅ Mejor modelo guardado.")

            fid.reset()
            is_metric.reset()

print("\\n✅ Entrenamiento WGAN-GP completo para todas las clases.")


\n🚀 Clase: G


c:\Users\34629\TFG\env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
100%|██████████| 17/17 [03:46<00:00, 13.33s/it]


[Epoch 1] FID: 420.75, IS: 1.00
✅ Mejor modelo guardado.


100%|██████████| 17/17 [03:23<00:00, 11.96s/it]


[Epoch 2] FID: 401.13, IS: 1.00
✅ Mejor modelo guardado.


...

[Epoch 45] FID: 464.67, IS: 1.39


100%|██████████| 61/61 [04:14<00:00,  4.18s/it]


[Epoch 46] FID: 449.52, IS: 1.49


100%|██████████| 61/61 [04:16<00:00,  4.20s/it]


RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 10141952 vs 10141840